# Análisis exploratorio de datos

## Importación de librerías

In [120]:
!pip install folium

   ---------------------------------------- 0.0/108.9 kB ? eta -:--:--
   ---------------------------------------- 108.9/108.9 kB 6.2 MB/s eta 0:00:00


In [121]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import os
import plotly.express as pex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import folium  
from   folium.plugins import HeatMap

## Preliminar

In [ ]:

path_file = r'C:\Users\danie\OneDrive\Documents\Bootcamp_Proyecto\stations.parquet' 
Sta = pd.read_parquet(path_file)
Sta
path_file = r'C:\Users\danie\OneDrive\Documents\Bootcamp_Proyecto\rain.parquet' 
Rain = pd.read_parquet(path_file)
Rain = Rain.sort_values(['Sta','Date'])
Rain


In [9]:
#Obtener un df donde cada columna muestre la info de los deptos

fecha_columna = 'FechaObservacion' 

In [ ]:
for file in files:
    file_path = os.path.join(folder_path, file)

    df = pd.read_parquet(file_path)
    
    rows, cols = df.shape
    
    if fecha_columna in df.columns:
       
        df[fecha_columna] = pd.to_datetime(df[fecha_columna], errors='coerce')

       
        fecha_minima = df[fecha_columna].min()
        fecha_maxima = df[fecha_columna].max()
        
        print(f"Archivo: {file} - Filas: {rows}, Columnas: {cols}")
        print(f"Fecha mínima: {fecha_minima}")
        print(f"Fecha máxima: {fecha_maxima}\n")
    else:
        print(f"Archivo: {file} - No se encontró la columna '{fecha_columna}'\n")

dataframes = {}


In [ ]:

for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    nombre_ciudad = file.split('_')[0]  # Extraer el nombre de la ciudad del nombre del archivo
    dataframes[nombre_ciudad] = df
dataframes={}


In [ ]:
resultados_agrupados = {}
for file in files:
    file_path = os.path.join(folder_path, file)
    
    df = pd.read_parquet(file_path)
    
    df['FechaObservacion'] = pd.to_datetime(df['FechaObservacion'], errors='coerce')
    
    
    df['Year'] = df['FechaObservacion'].dt.year
    df['Month'] = df['FechaObservacion'].dt.month
    
    
    df_grouped = df.groupby(['Year', 'Month','Departamento'], observed=False)['ValorObservado'].agg(['sum', 'mean']).reset_index()
    
    df_grouped.columns = ['Año', 'Mes', 'Departamento','Valor Total', 'Valor Promedio']
    
    # Guardar los resultados en el diccionario
    resultados_agrupados[nombre_ciudad] = df_grouped


In [ ]:
# Llamar al DataFrame de Bogotá
df_bogota =  resultados_agrupados[nombre_ciudad]
df_bogota

In [ ]:
### 1

df_part1 = pd.DataFrame()
for i in namesFiles:
    path_file = path + '\\' + i
    df_dept2 = pd.read_parquet(
                                path_file
                            )
    df_part2 = pd.concat([df_part2,df_dept2], ignore_index=True)
    del df_dept2

df_part1

In [ ]:
### EDA Bogota2
file_bogota=path+r'\rain_bogota2.parquet'
df_bogota = pd.read_parquet(file_bogota)
df_bogota.describe()
df_bogota.describe(include=['category'])

print(df_bogota['Departamento'].value_counts())
print(df_bogota['NombreEstacion'].value_counts())

In [ ]:

df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month

In [ ]:
year_total = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .sum()
    .reset_index()
)
year_total
year_mean = (
    df_bogota
    .groupby(["Year"])["ValorObservado"]
    .mean()
    .reset_index()
)
year_mean

In [ ]:
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Año'])
fig.add_trace(
    go.Scatter(x=year_total["Year"], y = year_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)

In [ ]:
month_total = (
    df_bogota
    .groupby(["Month"])["ValorObservado"]
    .sum()
    .reset_index()
)
month_total
fig = make_subplots(rows=1, cols=1
                    #  , row_heights =[0.7, 0.3]
                    # , shared_xaxes = True
                    ,subplot_titles = ['Meses'])
fig.add_trace(
    go.Scatter(x=month_total["Month"], y = month_total["ValorObservado"]
               , mode='lines+markers' # selcciona una combinación entre líneas y puntos
               , name = "Observado"
               ,legendgroup = '1'
               ),
)


In [ ]:
df_bogota['Year'] = df_bogota['FechaObservacion'].dt.year
df_bogota['Month'] = df_bogota['FechaObservacion'].dt.month
df_grouped = df_bogota.groupby(['Year', 'Month'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month', 'Valor Total', 'Valor Promedio']

df_grouped
df_grouped.max()


In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Total', data=df_grouped, marker='o')
plt.title('Distribución del Valor Total Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Total')
plt.show()
plt.figure(figsize=(12, 6))
sns.lineplot(x='Month', y='Valor Promedio', data=df_grouped, marker='o')
plt.title('Distribución del Valor Promedio Observado por Mes')
plt.xlabel('Mes')
plt.ylabel('Valor Promedio')
plt.show()
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Total', data=df_grouped, palette='Blues_d')
plt.title('Distribución del Valor Total Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Total')
plt.show()
plt.figure(figsize=(12, 6))
sns.barplot(x='Year', y='Valor Promedio', data=df_grouped)
plt.title('Distribución del Valor Promedio Observado por Año')
plt.xlabel('Año')
plt.ylabel('Valor Promedio')
plt.show()


In [ ]:
df_pivot = df_grouped.pivot(index='Year', columns='Month', values='Valor Total')


In [ ]:

# Heatmap para visualizar el valor total observado por año y mes
plt.figure(figsize=(12, 6))
sns.heatmap(df_pivot, cmap='coolwarm',annot=True,fmt='.0f')
plt.title('Distribución del Valor Total Observado por Año y Mes')
plt.show()


In [ ]:
df_grouped = df_bogota.groupby(['Year', 'Month','Latitud','Longitud'])['ValorObservado'].agg(['sum', 'mean']).reset_index()

df_grouped.columns = ['Year', 'Month','Latitud','Longitud', 'Valor Total', 'Valor Promedio']

df_grouped
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Latitud', y='Longitud', hue='Valor Total', data=df_grouped, palette='coolwarm')
plt.title('Distribución geográfica de Valor Observado')
plt.show

### 23 últimos

Cantidad de estaciones
X = Rain.loc[Rain['Sta']==0, 'Date'].values
Y = Rain.loc[Rain['Sta']==0, 'Rain'].values

plt.plot(X,Y)
plt.show()

## Exploratory Analysis

In [ ]:
#path_file=r"daniel...\Data\CleanDatasets3 - Merged"

In [122]:
path_file=r"C:\Users\Josue Florez\Documents\Maria Angelica\Proyectos\Proyecto_bootcamp\Data\CleanDatasets3 - Merged\stations.parquet"

In [124]:
df_rain=pd.read_parquet(path_file)

In [125]:
df_rain.shape

(894, 7)

In [126]:
df_rain.head()

,Sta,CodSta,NameSta,Lat,Long,Dept,City
0,0,11017020,PR CHOCO: BAGADO,5.412,-76.418,CHOCÓ,BAGADÓ
1,1,11025501,CARMEN DE ATRATO - AUT,5.889,-76.145,CHOCÓ,EL CARMEN
2,2,11027030,EL SIETE - AUT,5.862,-76.152,CHOCÓ,EL CARMEN
3,3,11027070,BORAUDO,5.515,-76.576,CHOCÓ,LLORÓ
4,4,11030010,PR CHOCO: CERTEGUI,5.377,-76.612,CHOCÓ,CÉRTEGUI


In [135]:
df_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Sta      894 non-null    int64   
 1   CodSta   894 non-null    uint64  
 2   NameSta  894 non-null    object  
 3   Lat      894 non-null    float64 
 4   Long     894 non-null    float64 
 5   Dept     894 non-null    category
 6   City     894 non-null    category
dtypes: category(2), float64(2), int64(1), object(1), uint64(1)
memory usage: 59.0+ KB


In [136]:
df_rain.isna().sum()

Sta        0
CodSta     0
NameSta    0
Lat        0
Long       0
Dept       0
City       0
dtype: int64

In [137]:
df_rain.duplicated().sum()


0

In [138]:
df_rain.drop_duplicates()

,Sta,CodSta,NameSta,Lat,Long,Dept,City
0,0,11017020,PR CHOCO: BAGADO,5.412,-76.418,CHOCÓ,BAGADÓ
1,1,11025501,CARMEN DE ATRATO - AUT,5.889,-76.145,CHOCÓ,EL CARMEN
2,2,11027030,EL SIETE - AUT,5.862,-76.152,CHOCÓ,EL CARMEN
3,3,11027070,BORAUDO,5.515,-76.576,CHOCÓ,LLORÓ
4,4,11030010,PR CHOCO: CERTEGUI,5.377,-76.612,CHOCÓ,CÉRTEGUI
...,...,...,...,...,...,...,...
889,889,5205500123,LAS IGLESIAS - AUT,1.055,-77.270,NARIÑO,PASTO
890,890,5311500056,UNIVERSIDAD DEL PACIFICO - AUT,3.848,-76.987,VALLE DEL CAUCA,BUENAVENTURA
891,891,5311500121,LA CUMBRE - AUT,3.645,-76.565,VALLE DEL CAUCA,LA CUMBRE
892,892,5311500149,COLEGIO VASCO NUÑEZ DE BALBOA,3.884,-77.049,VALLE DEL CAUCA,BUENAVENTURA


In [87]:
df_rain.duplicated().sum()


45835

In [102]:
df_rain.describe()

,FechaObservacion,ValorObservado
count,1091266,1.091266e+06
mean,2024-04-02 13:51:32.503825408,1.821802e+01
min,2023-09-11 01:00:00,0.000000e+00
25%,2023-12-04 12:00:00,1.200000e+01
50%,2024-03-27 11:20:00,1.697331e+01
75%,2024-08-07 06:00:00,2.460000e+01
max,2024-10-16 12:50:00,4.920000e+01
std,NaN,7.461070e+00


In [127]:
frecuencia_Sta=df_rain["Sta"].value_counts()
print(frecuencia_Sta)

Sta
0      1
600    1
589    1
590    1
591    1
      ..
301    1
302    1
303    1
304    1
893    1
Name: count, Length: 894, dtype: int64


In [82]:
frecuencia_rain=df_rain["ValorObservado"].value_counts()
print(frecuencia_rain)

ValorObservado
25.799999    4460
25.900000    4366
25.600000    4293
24.900000    4286
25.500000    4255
             ... 
14.055200       1
14.080590       1
25.994940       1
19.594061       1
11.358420       1
Name: count, Length: 442661, dtype: int64


In [83]:
frecuencia_date=pd.DataFrame(df_rain["FechaObservacion"].value_counts())
print(frecuencia_date)

                     count
FechaObservacion          
2024-09-05 12:00:00    401
2024-09-05 05:00:00    400
2024-09-05 01:00:00    399
2024-09-05 03:00:00    398
2024-09-08 04:00:00    397
...                    ...
2024-05-29 06:16:00      1
2024-05-29 05:48:00      1
2024-05-29 09:04:00      1
2024-05-29 07:17:00      1
2024-05-29 06:27:00      1

[24704 rows x 1 columns]


In [ ]:
df_rain

In [168]:
folium_map = folium.Map(location=[4.788056, -72.460000], zoom_start=6,tiles="Cartodb dark_matter")
for i in range(0,893):
    marker = folium.CircleMarker(location=[df_rain["Lat"][i],df_rain["Long"][i]],radius=3,
    color="cornflowerblue",
    stroke=False,
    fill=True,
    fill_opacity=0.6,
    opacity=1,
    popup="{}".format(df_rain["NameSta"][i]))
    marker.add_to(folium_map)
folium_map

In [103]:
path_file=r"C:\Users\Josue Florez\Downloads\weatherAUS.csv\weatherAUS.csv"

In [104]:
df=pd.read_csv(path_file)

In [105]:
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [111]:
df_Albury=df[df["Location"]=="MountGambier"]
df_Albury

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
99513,2008-12-01,MountGambier,6.6,18.7,4.0,5.0,10.6,WNW,61.0,WNW,...,73.0,39.0,1009.4,1008.9,3.0,3.0,13.4,17.8,Yes,No
99514,2008-12-02,MountGambier,8.9,18.1,0.0,5.4,3.6,W,50.0,NW,...,74.0,49.0,1009.8,1008.9,8.0,8.0,14.4,16.2,No,No
99515,2008-12-03,MountGambier,10.1,19.3,1.0,3.2,12.0,WSW,43.0,WSW,...,73.0,47.0,1011.9,1013.9,5.0,6.0,13.6,18.2,No,No
99516,2008-12-04,MountGambier,5.9,21.2,0.0,4.4,1.6,NNE,50.0,ENE,...,69.0,39.0,1014.8,1009.5,8.0,8.0,13.7,20.5,No,Yes
99517,2008-12-05,MountGambier,13.6,19.3,12.8,3.8,1.8,WNW,63.0,NW,...,86.0,65.0,1005.7,1006.9,8.0,7.0,16.1,18.7,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102548,2017-06-21,MountGambier,8.7,13.2,2.0,NaN,NaN,S,20.0,NaN,...,98.0,82.0,1029.2,1028.3,NaN,8.0,11.1,12.0,Yes,No
102549,2017-06-22,MountGambier,7.0,15.7,0.0,NaN,NaN,N,31.0,N,...,92.0,67.0,1027.0,1022.4,8.0,NaN,10.0,14.9,No,No
102550,2017-06-23,MountGambier,5.4,16.2,0.0,NaN,NaN,N,37.0,NNW,...,90.0,70.0,1017.0,1014.4,8.0,4.0,6.4,14.9,No,Yes
102551,2017-06-24,MountGambier,4.8,12.3,2.2,NaN,NaN,W,28.0,NNW,...,93.0,70.0,1020.0,1018.0,5.0,8.0,8.2,11.5,Yes,No


In [108]:
frecuencia_ciudad=(df["Location"].value_counts())
print(frecuencia_ciudad)

Location
Canberra            3436
Sydney              3344
Darwin              3193
Melbourne           3193
Brisbane            3193
Adelaide            3193
Perth               3193
Hobart              3193
Albany              3040
MountGambier        3040
Ballarat            3040
Townsville          3040
GoldCoast           3040
Cairns              3040
Launceston          3040
AliceSprings        3040
Bendigo             3040
Albury              3040
MountGinini         3040
Wollongong          3040
Newcastle           3039
Tuggeranong         3039
Penrith             3039
Woomera             3009
Nuriootpa           3009
Cobar               3009
CoffsHarbour        3009
Moree               3009
Sale                3009
PerthAirport        3009
PearceRAAF          3009
Witchcliffe         3009
BadgerysCreek       3009
Mildura             3009
NorfolkIsland       3009
MelbourneAirport    3009
Richmond            3009
SydneyAirport       3009
WaggaWagga          3009
Williamtown     